# Wine Quality Classification

The notebook is intended to develop & validate a model for multi-class classification of the Wine Quality.

In [ ]:
# Import Standard Libraries
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore

In [ ]:
# Define Seaborn theme parameters
theme_parameters =  {
    'axes.spines.right': False,
    'axes.spines.top': False,
    'grid.alpha':0.3,
    'figure.figsize': (16, 6),
    'font.family': 'Andale Mono',
    'axes.titlesize': 24,
    'figure.facecolor': '#E5E8E8',
    'axes.facecolor': '#E5E8E8'
}

# Set the theme
sns.set_theme(style='whitegrid',
              palette=sns.color_palette('deep'), 
              rc=theme_parameters)

In [ ]:
# Notebook's variables
train_data_path = os.path.join('./../../data/S3E5/wine_quality_train.csv')
test_data_path = os.path.join('./../../data/S3E5/wine_quality_test.csv')

# Read Data

In [ ]:
# Read train and test data
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Exploratory Data Analysis

## Train Feature & Label Distribution

In [ ]:
# Plot the histograms of each feature
figure, ax = plt.subplots(3, 4, figsize=(16, 9))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' column)
for index, column_name in enumerate(train_data.columns[1:]):
    
    # Plot data
    sns.histplot(data=train_data[column_name], 
                 ax=ax[index])
    
    ax[index].set_title(column_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
plt.tight_layout()

The following features have a skewed distribution:
- `fixed acidity`
- `citric acid`
- `residual sugar`
- `free sulfur dioxide`
- `total sulfur dioxide`
- `sulphates`
- `alcohol`

It would be useful to use the Z-Score Outliers Filter.
<br>

In addition, it is possible to see that the label classes 3, 4 and 8 do not have a lots of records. That is an imbalanced data problem. Consider to use a Stratified K-Fold during the training of the model.

## Train Feature Distribution per Label

In [ ]:
# Plot the box plot of each feature per label
figure, ax = plt.subplots(3, 4, figsize=(16, 12))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' column and 'quality' column)
for index, column_name in enumerate(train_data.columns[1:-1]):
    
    # Plot data
    sns.boxplot(data=train_data,
                x='quality',
                y=column_name,
                ax=ax[index])
    
        
# Remove the empty subplot
figure.delaxes(ax[-1])

plt.suptitle('Feature Distrubtion per Label', fontsize=14)
plt.tight_layout()
plt.show()

There is a positive non-linear relationship between the following features and the `Quality`:
- `sulphates`
- `alcohol`

Thre is a negative non-linear relationship between the following features the the `Quality`:
- `voltatile acidity`
- `density`

## Train vs Test Feature & Label Distribution

In [ ]:
# Plot the KDE of each feature
figure, ax = plt.subplots(3, 4, figsize=(16, 12))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' and 'quality' columns)
for index, column_name in enumerate(train_data.columns[1:-1]):
    
    # Plot data
    sns.kdeplot(data=train_data[column_name],
                label='Train',
                ax=ax[index])
    
    sns.kdeplot(data=test_data[column_name],
                label='Test',
                ax=ax[index])
    
    ax[index].set_title(column_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
    # Retrieve legend information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()
    
# Remove the empty subplot
figure.delaxes(ax[-1])

# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=2)
    
plt.tight_layout()

There are no strong differences between the feature distribution of the train set and the test set.

## Count Outliers with the Z-Score across Quality Classes

In [ ]:
# Compute the Z-Score for the feature columns across 'quality' classes
z_scores = train_data.iloc[:, 1:-1].groupby(train_data['quality'], 
                                            group_keys=True).apply(zscore)

In [ ]:
# Consider as an 'outlier' every records with a Z-Score bigger than 2 SDs in absolute value terms
outlisers = z_scores.abs().ge(2).groupby(z_scores.index.get_level_values(0)).sum()